In [ ]:
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import azure.functions as func
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import json
import mysql.connector
import requests
from mysql.connector import errorcode
import ssl
import pymysql
import datetime

In [ ]:
KEY_FILE_LOCATION = "client_secrets.json"
SCOPES = ["https://www.googleapis.com/auth/analytics.readonly"]
 
startDate ='2020-01-01'
endDate='2023-05-31'
dimensions = ['ga:year','ga:month','ga:mobileDeviceInfo']
metrics = ['ga:users', 'ga:newUsers','ga:sessions','ga:sessionsPerUser','ga:pageviews','ga:pageviewsPerSession','ga:avgSessionDuration','ga:bounceRate']

In [ ]:
def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build("analyticsreporting", "v4", credentials=credentials)

    return analytics


Analytics = initialize_analyticsreporting()
 
def request(pageToken ='undefined'):
    response = Analytics.reports().batchGet(
        # TODO: Validate values and prevent false query
        body={
            "reportRequests": [
                {
                    "viewId": '178838015',
                    "dateRanges": {'startDate': startDate, 'endDate': endDate},
                    "metrics": [{'expression': expression} for expression in metrics],
                    "dimensions": [{'name': name} for name in dimensions],
                     "pageSize": 100000,
                    "pageToken": pageToken
            }]
        }
    ).execute()
    return response

In [ ]:
from ga import manipulate (you can import below function from any python code like ga.py)

In [ ]:
def manipulate(response):
    for report in response.get('reports', []):
        column_h = report.get('columnHeader', {})
        dimension_h = column_h.get('dimensions', [])
        metric_h = [i.get('name', {}) for i in column_h.get('metricHeader', {}).get('metricHeaderEntries', [])]
        final_rows = []

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            metrics = row.get('metrics', [])[0].get('values', {})
            row_obj = {}

            for header, dimension in zip(dimension_h, dimensions):
                row_obj[header] = dimension

            for metric_headers, metric in zip(metric_h, metrics):
                row_obj[metric_headers] = metric

            final_rows.append(row_obj)

    df = pd.DataFrame(final_rows)
    return df


In [ ]:
def next_records():
  records =[]
  response= request()
  nextPageToken = response.get("reports")[0].get('nextPageToken', None)
  df = manipulate(response)
  records.append(df)
  while nextPageToken != None:
    response = request(nextPageToken)
    df = manipulate(response)
    records.append(df)
    nextPageToken = response.get("reports")[0].get('nextPageToken', None)
  df = pd.concat(records).reset_index(drop=True)
  df.columns = df.columns.str.replace(r'ga:', '')
  return df

ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
 
config={ 
'host':'host',
'database':'databse',
'user':'user',
'password':'password',
'ssl':ssl_context
} 

In [ ]:
def azure_sql_insert(df):
    try:
        conn = pymysql.connect(**config)
        print("Connection established")
        cursor = conn.cursor()
        insert_query = "INSERT INTO tablenames(year,month,interestOtherCategory,users,newUsers,sessions,sessionsPerUser,pageviews,pageviewsPerSession,avgSessionDuration,bounceRate) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        data = [tuple(row) for row in df.to_numpy()]
        cursor.executemany(insert_query, data)
        conn.commit()
        cursor.close()
        conn.close()
        print("Data inserted successfully")
        return f"Connection established"

    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            error_message = "Something is wrong with the user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            error_message = "Database does not exist"
        else:
            error_message = str(err)
        print(error_message)
        return error_message

In [ ]:
data=next_records()
azure_sql_insert(data)